In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *

Lassi


In [7]:
class Graph:
    def __init__(self):
        self.adjacency_list = {}
        self.edge_list = set()

    def add_node(self, node):
        if node not in self.adjacency_list:
            self.adjacency_list[node] = set()

    def add_edge(self, node1, node2):
        self.add_node(node1)
        self.add_node(node2)
        self.adjacency_list[node1].add(node2)
        self.adjacency_list[node2].add(node1)
        self.edge_list.add((node1,node2))
        self.edge_list.add((node2,node1))

    def remove_edge(self, node1, node2):
        if node1 in self.adjacency_list and node2 in self.adjacency_list:
            if node2 in self.adjacency_list[node1]:
                self.adjacency_list[node1].remove(node2)
                self.adjacency_list[node2].remove(node1)
        self.edge_list.remove((node1,node2))
        self.edge_list.remove((node2,node1))

    def get_nodes_by_degree(self):
        nodes_with_degrees = [(node, len(neighbors)) for node, neighbors in self.adjacency_list.items()]
        return [node for node, _ in sorted(nodes_with_degrees, key=lambda x: x[1])]

    def is_edge(self,node1,node2):
        return (node1,node2) in self.edge_list and (node2,node1) in self.edge_list

In [8]:
def check_create_edge(flow1,flow2):
    flow1_sender = flow1[2]
    flow1_receiver = flow1[3]
    flow2_sender = flow2[2]
    flow2_receiver = flow2[3]

    if flow1_sender == flow2_sender or flow1_receiver == flow2_receiver: # No concurrency in this case
        return 1
    elif flow1_sender == flow2_receiver and flow2_sender == flow1_receiver:
        return calculate_relative_intereference1() >= interference_threshold
    elif flow1_sender == flow2_receiver:
        return max(calculate_relative_intereference1(),calculate_relative_intereference2(flow2_sender,flow1_receiver)) >= interference_threshold
    elif flow2_sender == flow1_receiver:
        return max(calculate_relative_intereference1(),calculate_relative_intereference2(flow1_sender,flow2_receiver)) >= interference_threshold
    else:
        return max(calculate_relative_intereference2(flow1_sender,flow2_receiver),calculate_relative_intereference2(flow2_sender,flow1_receiver)) >= interference_threshold
    

In [9]:
for current_time_slot in range(number_of_time_slots):
    time = current_time_slot * slot_duration
    # Creating a graph for each time slot
    graph = Graph()
    # Comparing flow pair at a time for graph connection
    for flow1 in flows:
        flow_with_time1 = [flow1[0].calculate_position(time), flow1[1].calculate_position(time), time]
        flow_with_time1 = str(flow_with_time1)
        relay_type1, candidate_set1, random_candidate1 = flows_candidate_relay_set_pairs[flow_with_time1]
        # If flow 1 is not relayed OR there is no relaying candidate for flow, no need to include it in graph
        if relay_type1 == 'dont_relay' or random_candidate1 == -1:
            continue
        for flow2 in flows:
            if flow1.all() == flow2.all():
                continue
            flow_with_time2 = [flow2[0].calculate_position(time), flow2[1].calculate_position(time), time]
            flow_with_time2 = str(flow_with_time2)
            relay_type2, candidate_set2, random_candidate2 = flows_candidate_relay_set_pairs[flow_with_time2]
            # If flow 2 is not relayed OR there is no relaying candidate for flow, no need to include it in graph
            if relay_type2 == 'dont_relay' or random_candidate2 == -1:
                continue
            if random_candidate1 == random_candidate2 or check_create_edge(flow1,flow2):
                graph.add_edge(tuple(flow1),tuple(flow2))
    # Now we make groups in graph so that each group contains flows that are not in contention  
    groups = []
    # key = group_no, value = list of group members
    degree_sorted_node_list = graph.get_nodes_by_degree()
    for node in degree_sorted_node_list:
        found_group = False
        for group in groups:
            no_edge = True
            for flow in group:
                if graph.is_edge(node,flow):
                    no_edge = False
                    break
            if no_edge == True:
                group.append(node)
                found_group = True
                break
        if found_group == False:
            groups.append([node])
            
            

4